In [ ]:
import pandas as pd
import gurobipy as gb
from gurobipy import GRB
import numpy as np
import warnings
import time
import asyncio
from matplotlib import pyplot as plt
import geopandas as gpd

# Load data
df_main = pd.read_excel("C:\\Users\\smmashai\\Downloads\\Thesis\\master-shaik-main\\classic-model\\SimData\\Nodes old RTF.xlsx")

Nodes = df_main.iloc[:, [0, 2, 3]]
Supply = df_main.iloc[:, [4, 5, 6, 7, 8, 9, 10]]
Demand = df_main.iloc[:, [11, 12, 13, 14, 15, 16, 17]]

Materials = ["Mixed", "Foam", "Briq", "pyrOil", "ANL", "P-TOL", "NaphtaSub"]
tech = ["CF", "RTF", "CPF", "DPF"]

yield_array = [[-1, 0.01, 0, 0, 0, 0, 0],
                [0, -1, 0.8, 0, 0, 0, 0],
                [0, 0, -1, 0.65, 0, 0, 0],
                [0, 0, 0, -1,  0.47, 0.25, 0.2]]

actual_marketprice = [0, 0, 0, 0, 1495, 1500, 380]
m = 1.5
marketprice = [m * price for price in actual_marketprice]
final_og = 10
opt_og = 0.1
facility_capacity_list = [[300,150,100,80,50], [70, 40,30,20,10], [100,80,50,25,10], [45,60,35,90,15]]
#facility_capacity_list = [[960, 760, 560, 360, 160], [70, 55, 40, 25, 10], [60, 50, 40, 30, 20], [40, 32, 25, 17, 10]]
#facility_capacity_list = [[2000, 1650, 1300, 950, 600], [100, 80, 60, 40, 20], [180, 160, 140, 120, 100], [140, 120, 100, 80, 60]]

# Reference costs and capacities
refFixedCF = 0#1.45  # MM euro
refFixedRTF = 2.88#12.8  # MM euro
refFixedCPF = 52.5  # MM euro
refFixedDPF = 47.7  # MM euro
refCapacityCF = 100  # ton/day
refCapacityRTF = 100  # ton/day
refCapacityCPF = 300  # ton/day
refCapacityDPF = 192  # ton/day
capexScalingFactor = 0.6
rate = 0.1
period = 10

# Overhead costs
overheadCF = 0.267 * 0.01  # MM euro/year
overheadRTF = 0.817  # MM euro/year
overheadCPF = 3.66  # MM euro/year
overheadDPF = 3.52  # MM euro/year
workingDays = 330

# Function to calculate facility costs
def calc_facility_cost(k):
    tot_cost_CF = 0 #refFixedCF * ((facility_capacity_list[0][k] / refCapacityCF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_RTF = refFixedRTF * ((facility_capacity_list[1][k] / refCapacityRTF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_CPF = refFixedCPF * ((facility_capacity_list[2][k] / refCapacityCPF) ** capexScalingFactor) * (10 ** 6)
    tot_cost_DPF = refFixedDPF * ((facility_capacity_list[3][k] / refCapacityDPF) ** capexScalingFactor) * (10 ** 6)
    Annual_cost_CF = rate * tot_cost_CF / (1 - (1 + rate) ** (-period))
    Annual_cost_RTF = rate * tot_cost_RTF / (1 - (1 + rate) ** (-period))
    Annual_cost_CPF = rate * tot_cost_CPF / (1 - (1 + rate) ** (-period))
    Annual_cost_DPF = rate * tot_cost_DPF / (1 - (1 + rate) ** (-period))
    return [Annual_cost_CF, Annual_cost_RTF, Annual_cost_CPF, Annual_cost_DPF]

facility_total_cost = [calc_facility_cost(k) for k in range(len(facility_capacity_list[0]))]
facility_total_cost = list(map(list, zip(*facility_total_cost)))

facility_var_costs = [0.15, 46, 68, 102]  # Updated variable costs

# Calculate fixed operating costs (fOPEX)
def calc_fixed_operating_cost(k):
    fOPEX_CF = overheadCF * ((facility_capacity_list[0][k] / refCapacityCF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_RTF = overheadRTF * ((facility_capacity_list[1][k] / refCapacityRTF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_CPF = overheadCPF * ((facility_capacity_list[2][k] / refCapacityCPF) ** capexScalingFactor) * (10 ** 6) 
    fOPEX_DPF = overheadDPF * ((facility_capacity_list[3][k] / refCapacityDPF) ** capexScalingFactor) * (10 ** 6) 
    return [fOPEX_CF, fOPEX_RTF, fOPEX_CPF, fOPEX_DPF]

fixed_operating_costs = [calc_fixed_operating_cost(k) for k in range(len(facility_capacity_list[0]))]
fixed_operating_costs = list(map(list, zip(*fixed_operating_costs)))
transport_costs = [0.14124335003280253, 0.2600782146695747, 0.057932422317647767, 0.03245140669834265, 0.025961125358674116, 0.025961125358674116, 0.025961125358674116]

# Function to calculate distances using Haversine formula
def distanceHaversine(city, lat, lon):
    df_dist = pd.DataFrame([[0] * (len(city) + 1)] * (len(city) + 1))
    city_names = city
    lat_df = lat
    lon_df = lon
    radius = 6371

    for i in range(len(city)):
        df_dist.iloc[i + 1, 0] = city_names.iloc[i]
        df_dist.iloc[0, i + 1] = city_names.iloc[i]
        for j in range(len(city)):
            lat1 = lat_df.iloc[i]
            lon1 = lon_df.iloc[i]
            lat2 = lat_df.iloc[j]
            lon2 = lon_df.iloc[j]
            delta_lat = np.radians(lat2 - lat1)
            delta_lon = np.radians(lon2 - lon1)

            a = (np.sin(delta_lat / 2)) ** 2 + np.cos(np.radians(lat1)) * np.cos(np.radians(lat2)) * (np.sin(delta_lon / 2)) ** 2
            c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
            d = radius * c
            df_dist.iloc[i + 1, j + 1] = d
    return df_dist

warnings.simplefilter(action='ignore', category=FutureWarning)

df_dist_withcitynames = distanceHaversine(Nodes.iloc[:, 0], Nodes.iloc[:, 1], Nodes.iloc[:, 2])
df_distances = df_dist_withcitynames.iloc[1:, 1:].apply(pd.to_numeric)

multipliers = {0: 0.2, 1: 2, 2: 2, 3: 2}
num_nodes = len(Nodes)
num_mat = len(Materials)
num_techs = len(tech)
num_cap = len(facility_capacity_list[0])

percentages = [1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100]  # Adjust as needed 1,5,10,15,20,25,30,35,40,45,50,55,60,65,70,75,80,85,90,95,100
tech_decision_lb = []
tech_capacity_lb = []
N_values_lb = []
execution_times_lb = []
objective_values_lb = []
N_values_ub = []
execution_times_ub = []
objective_values_ub = []
N_ub = [int(num_nodes * i / 100) for i in percentages]
N_lb = [int(num_nodes *num_nodes * i / 100) for i in percentages]
results = []

#####some error in lb optimization
# Asynchronous function to solve LB problem
import gurobipy as gb
import numpy as np
import time

def solve_lb(N_value_lb):
    all_execution_times_lb = []
    all_objective_values_lb = []

    for _ in range(13):
        N_values_lb.append(N_value_lb)
        lb_size_of_sample = N_lb[N_lb.index(N_value_lb)]
        print("lb function")
        A_LB = np.arange(1, num_nodes * num_nodes + 1)

        # Sample lb_size_of_sample elements without replacement from A_LB
        S_sample = np.random.choice(A_LB, lb_size_of_sample, replace=False)

        # Initialize matrix C and fill it based on the sampled indices
        C = np.zeros((num_nodes, num_nodes))
        for k in S_sample:
            i = (k - 1) // num_nodes
            j = (k - 1) % num_nodes
            C[i, j] = 1

        # Initialize matrix CC with dimensions (num_nodes, num_nodes) and fill it with zeros
        CC = np.zeros((num_nodes, num_nodes))
        for i in range(len(Nodes)):
            for j in range(len(Nodes)):
                CC[i, j] = C[i, j]

        global last_tech_capacity_lb, last_tech_decision_lb
        start_time_lb = time.time()
        full_problem_model_lb = gb.Model("FullProblem lb")
        full_problem_model_lb.setParam('MIPGap', opt_og)

        flow = full_problem_model_lb.addVars(((i, j, m) for i in range(num_nodes) for j in range(num_nodes) for m in range(num_mat) if CC[i, j] == 1), vtype=gb.GRB.CONTINUOUS)
        demand_node = full_problem_model_lb.addVars(num_nodes, num_mat, vtype=gb.GRB.CONTINUOUS)
        supply_node = full_problem_model_lb.addVars(num_nodes, num_mat, vtype=gb.GRB.CONTINUOUS)
        tech_capacity = full_problem_model_lb.addVars(num_nodes, num_techs, vtype=gb.GRB.CONTINUOUS)
        operating_cost = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)
        fixed_operating_cost = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)
        capital_cost = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)
        transport_cost = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)
        tech_decision = full_problem_model_lb.addVars(num_nodes, num_techs, num_cap, vtype=gb.GRB.BINARY)

        for i in range(num_nodes):
            for t in range(num_techs):
                full_problem_model_lb.addConstr(tech_capacity[i, t] >= 0)
                full_problem_model_lb.addConstr(tech_capacity[i, t] <= gb.quicksum(tech_decision[i, t, c] * facility_capacity_list[t][c] for c in range(num_cap)))
                full_problem_model_lb.addConstr(1 >= gb.quicksum(tech_decision[i, t, c] for c in range(num_cap)))

        # Add objective function: maximize social welfare
        obj = full_problem_model_lb.addVar(vtype=gb.GRB.CONTINUOUS)

        # Constraints for supply, demand, and flow conservation
        for i in range(num_nodes):
            for m in range(num_mat):
                full_problem_model_lb.addConstr(supply_node[i, m] <= Supply.iloc[i, m])
                full_problem_model_lb.addConstr(demand_node[i, m] <= Demand.iloc[i, m])
                full_problem_model_lb.addConstr(
                    supply_node[i, m] + gb.quicksum(flow[j, i, m] for j in range(num_nodes) if CC[j, i] == 1) + gb.quicksum(yield_array[t][m] * tech_capacity[i, t] for t in range(num_techs)) == demand_node[i, m] + gb.quicksum(flow[i, j, m] for j in range(num_nodes) if CC[i, j] == 1))

        # Constraints for transportation cost, operating cost, and capital cost
        full_problem_model_lb.addConstr(
            transport_cost == gb.quicksum(transport_costs[m] * df_distances.iloc[i, j] * flow[i, j, m] for i in range(num_nodes) 
                                          for j in range(num_nodes) for m in range(num_mat) if CC[i, j] == 1))
        full_problem_model_lb.addConstr(
            operating_cost == gb.quicksum(facility_var_costs[t] * tech_capacity[i, t] for i in range(num_nodes) 
                                          for t in range(num_techs)))
        full_problem_model_lb.addConstr(
            fixed_operating_cost == gb.quicksum(tech_decision[i, t, c] * fixed_operating_costs[t][c] for i in range(num_nodes) 
                                                for t in range(num_techs) for c in range(num_cap)))
        full_problem_model_lb.addConstr(
            capital_cost == gb.quicksum(tech_decision[i, t, c] * facility_total_cost[t][c] for i in range(num_nodes) 
                                        for t in range(num_techs) for c in range(num_cap)))

        full_problem_model_lb.addConstr(
            obj == workingDays * gb.quicksum(marketprice[m] * demand_node[i, m] for i in range(num_nodes) for m in range(num_mat)) 
            - 2 * workingDays * transport_cost - workingDays * operating_cost - fixed_operating_cost - capital_cost)

        full_problem_model_lb.setObjective(obj, gb.GRB.MAXIMIZE)

        # Optimize the model
        full_problem_model_lb.optimize()
        end_time_lb = time.time()
        revenue = 0

        # Retrieve the values of demand_node variables
        for i in range(num_nodes):
            for m in range(num_mat):
                revenue += marketprice[m] * demand_node[i, m].X

        # Multiply by workingDays as per the objective function definition
        revenue *= workingDays

        # Sum tech capacities across all nodes for each technology
        tech_capacity_sums = {t: sum(tech_capacity[i, t].X for i in range(num_nodes)) for t in range(num_techs)}

        # Print the summed tech capacities for each technology
        for t in range(num_techs):
            print(f"Total capacity for technology {t}: {tech_capacity_sums[t]}")

        # Sum demands across all nodes for each material
        demand_sums = {m: sum(demand_node[i, m].X for i in range(num_nodes)) for m in range(num_mat)}

        # Print the summed demands for each material
        for m in range(num_mat):
            print(f"Total demand for material {m}: {demand_sums[m]}")

        a = operating_cost.X
        b = transport_cost.X
        c = capital_cost.X
        d = fixed_operating_cost.X

        # Print the revenue and costs
        print(f"Revenue: {revenue}")
        print(f"Operating cost: {a}")
        print(f"Transport cost: {b}")
        print(f"Capital cost: {c}")
        print(f"Fixed Operating cost: {d}")

        last_tech_capacity_lb = {(i, t): tech_capacity[i, t].X for i in range(num_nodes) for t in range(num_techs)}
        last_tech_decision_lb = {(i, t, c): tech_decision[i, t, c].X for i in range(num_nodes) for t in range(num_techs) for c in range(num_cap)}

        # Sum tech capacities across all nodes for each technology
        actual_capacity_used = {t: sum(tech_capacity[i, t].X for i in range(num_nodes)) for t in range(num_techs)}

        # Print the summed actual capacities used for each technology
        for t in range(num_techs):
            print(f"Actual capacity used for technology {t}: {actual_capacity_used[t]}")

        # Calculate and print the built capacity and percentage facility usage
        built_capacity = {t: sum(tech_decision[i, t, c].X * facility_capacity_list[t][c] for i in range(num_nodes) for c in range(num_cap)) for t in range(num_techs)}

        for t in range(num_techs):
            usage_percentage = (actual_capacity_used[t] / built_capacity[t]) * 100 if built_capacity[t] > 0 else 0
            print(f"Built capacity for technology {t}: {built_capacity[t]}")
            print(f"Percentage facility usage for technology {t}: {usage_percentage:.2f}%")

        execution_times_lb.append(end_time_lb - start_time_lb)
        objective_values_lb.append(full_problem_model_lb.ObjVal)
        all_execution_times_lb.append(end_time_lb - start_time_lb)
        all_objective_values_lb.append(full_problem_model_lb.ObjVal)

        results.append({
            'N_value': N_value_lb,
            'Type': 'LB',
            'Execution_Time': end_time_lb - start_time_lb,
            'Objective_Value': full_problem_model_lb.ObjVal
        })

    best_obj_val_lb = min(all_objective_values_lb)
    print("Best objective value from 10 runs:", best_obj_val_lb)

    return best_obj_val_lb

   


def solve_ub(N_value_ub):

    
    all_execution_times_ub = []
    all_objective_values_ub = []

    for _ in range(1):#print(S_sample)
        N_values_ub.append(N_value_ub)
        Ub_size_of_sample = N_ub[N_ub.index(N_value_ub)]
        print("UB function")
        A_UB = np.arange(1, num_nodes)
        # Sample lb_size_of_sample elements without replacement from A_UB
        S_sample = np.random.choice(A_UB, Ub_size_of_sample, replace=False)
        num_coarse_nodes = len(S_sample)
        df_distances_copy = df_distances.copy()
        classfication_problem = gb.Model("Nodeclassifier")
        classfication_problem.setParam('MIPGap', 0.1)
        node_class_decision = classfication_problem.addVars(num_coarse_nodes, num_nodes,vtype=GRB.BINARY)
        sum_class_distance = classfication_problem.addVar(vtype=GRB.CONTINUOUS)
        classfication_problem.addConstr(sum_class_distance == gb.quicksum(df_distances_copy.iloc[S_sample[i],j]*node_class_decision[i,j] for i in range(num_coarse_nodes) for j in range(num_nodes)))
        for j in range(num_nodes):    
            classfication_problem.addConstr(1==gb.quicksum(node_class_decision[i,j] for i in range(num_coarse_nodes)))
        classfication_problem.setObjective(sum_class_distance, gb.GRB.MINIMIZE)

        # Optimize the model
        classfication_problem.optimize()
        
        for i in range(num_coarse_nodes):
            for j in range(num_coarse_nodes):
                L = []
                R = []
            for m in range(num_nodes):
                if node_class_decision[i, m].x ==1:
                    L.append(m)
            for n in range(num_nodes):
                if node_class_decision[j, n].x ==1:
                    R.append(n)
            for mm in L:
                for nn in R:
                    if df_distances_copy.iloc[i,j]>=df_distances_copy.iloc[mm,nn]:
                        df_distances_copy.iloc[i,j]=df_distances_copy.iloc[mm,nn]
                        #print(i,j, "=",mm,nn,df_distances.iloc[mm,nn])
        #df_distances_copy.to_excel("final_distances.xlsx", index=False)
        
        start_time_ub = time.time()

        full_problem_model_UB = gb.Model("FullProblem")
        full_problem_model_UB.setParam('MIPGap', opt_og)
        

        flow = full_problem_model_UB.addVars(((i, j, m) for i in range(num_coarse_nodes) for j in range(num_coarse_nodes) for m in range(num_mat) ), vtype=gb.GRB.CONTINUOUS)
        demand_node = full_problem_model_UB.addVars(num_nodes, num_mat, vtype=GRB.CONTINUOUS)
        supply_node = full_problem_model_UB.addVars(num_nodes, num_mat, vtype=GRB.CONTINUOUS)
        tech_capacity = full_problem_model_UB.addVars(num_nodes, num_techs, vtype=GRB.CONTINUOUS)
        operating_cost = full_problem_model_UB.addVar(vtype=GRB.CONTINUOUS)
        fixed_operating_cost = full_problem_model_UB.addVar(vtype=GRB.CONTINUOUS)
        capital_cost = full_problem_model_UB.addVar(vtype=GRB.CONTINUOUS)
        transport_cost = full_problem_model_UB.addVar(vtype=GRB.CONTINUOUS)
        tech_decision = full_problem_model_UB.addVars(num_nodes, num_techs, num_cap, vtype=GRB.BINARY)

        for i in range(num_nodes):
            for t in range(num_techs):
                full_problem_model_UB.addConstr(tech_capacity[i, t] >= 0)
                full_problem_model_UB.addConstr(tech_capacity[i, t] <= gb.quicksum(tech_decision[i, t, c] * facility_capacity_list[t][c] for c in range(num_cap)))
                full_problem_model_UB.addConstr(1 >= gb.quicksum(tech_decision[i, t, c]  for c in range(num_cap)))

        # Add objective function: maximize social welfare
        obj = full_problem_model_UB.addVar(vtype=gb.GRB.CONTINUOUS)

        # Constraints for supply, demand, and flow conservation
        for i in range(num_nodes):
            for m in range(num_mat):
                full_problem_model_UB.addConstr(supply_node[i, m] <= Supply.iloc[i, m])
                full_problem_model_UB.addConstr(demand_node[i, m] <= Demand.iloc[i, m])
        #full_problem_model_UB.addConstr(gb.quicksum(supply_node[i, 0] for i in range(num_nodes)) >= 0.8*gb.quicksum(Supply.iloc[i, 0] for i in range(num_nodes)))
        
        for i in range(num_coarse_nodes):
            for m in range(num_mat):
                full_problem_model_UB.addConstr(
                    gb.quicksum(supply_node[j, m] for j in range(num_nodes) if node_class_decision[i, j].x == 1) +
                    gb.quicksum(flow[j, i, m] for j in range(num_coarse_nodes)) +
                    gb.quicksum(yield_array[t][m] * tech_capacity[j, t] for t in range(num_techs) for j in range(num_nodes) if node_class_decision[i, j].x == 1) ==
                    gb.quicksum(demand_node[j, m] for j in range(num_nodes) if node_class_decision[i, j].x == 1) +
                    gb.quicksum(flow[i, j, m]  for j in range(num_coarse_nodes)))

        # Constraints for transportation cost, operating cost, and capital cost
        full_problem_model_UB.addConstr(
            transport_cost == gb.quicksum(transport_costs[m] * df_distances_copy.iloc[S_sample[i], S_sample[j]] * flow[i, j, m] for i in range(num_coarse_nodes) for j in range(num_coarse_nodes) for m in range(num_mat)))
        full_problem_model_UB.addConstr(
            operating_cost == gb.quicksum(facility_var_costs[t] * tech_capacity[i, t] for i in range(num_nodes) for t in range(num_techs)))
        full_problem_model_UB.addConstr(
            fixed_operating_cost == gb.quicksum(tech_decision[i,t,c]*fixed_operating_costs[t][c] for i in range(num_nodes) 
                                            for t in range(num_techs) for c in range(num_cap)))
        full_problem_model_UB.addConstr(
            capital_cost == gb.quicksum(tech_decision[i,t,c]*facility_total_cost[t][c]  for i in range(num_nodes) 
                                            for t in range(num_techs) for c in range(num_cap)))#capital cost per year
        full_problem_model_UB.addConstr(
            obj == workingDays * gb.quicksum(marketprice[m] * demand_node[i, m] for i in range(num_nodes) for m in range(num_mat)) - 2*workingDays * transport_cost - workingDays * operating_cost -fixed_operating_cost- capital_cost)
        full_problem_model_UB.setObjective(obj, gb.GRB.MAXIMIZE)

        # Optimize the model
        full_problem_model_UB.optimize()

        end_time_ub = time.time()
        execution_time_ub = end_time_ub - start_time_ub
        print("Execution time:", execution_time_ub, "seconds")
        
        # Store the results of each run
        all_execution_times_ub.append(execution_time_ub)
        all_objective_values_ub.append(full_problem_model_UB.ObjVal)
        results.append({
            'N_value': N_value_ub,
            'Type': 'UB',
            'Execution_Time': execution_time_ub,
            'Objective_Value': full_problem_model_UB.ObjVal
        })

    # Store the best objective value from all runs
    best_obj_val = min(all_objective_values_ub)
    print("Best objective value from 10 runs:", best_obj_val)

    return best_obj_val

def optimality_gap(ub_obj, lb_obj):
    return ((ub_obj - lb_obj) / ub_obj) * 100


iteration_lb = 0
iteration_ub = 0
print(N_ub[iteration_ub])   
print(N_lb[iteration_lb])
ub_obj = solve_ub(N_ub[iteration_ub])
lb_obj = solve_lb(N_lb[iteration_lb])
prev_og = 10000
og = optimality_gap(ub_obj,lb_obj)
num_iteration_lb =0
num_iteration_ub =0
max_iter = len(percentages)+1
# Main optimization loop
while og>final_og and num_iteration_lb<max_iter and num_iteration_ub<max_iter:
    improvement = prev_og-og
    print("current improvement")
    print(improvement)
    if improvement<0.3:
        iteration_lb +=1
        print("lb")
        lb_obj = solve_lb(N_lb[iteration_lb])
        prev_og = og
        og = optimality_gap(ub_obj,lb_obj)
        num_iteration_lb +=1
        improvement = prev_og-og
        print("current og")
        print(og)
    else:
        print("ub")
        iteration_ub +=1
        ub_obj = solve_ub(N_ub[iteration_ub])
        prev_og = og
        og = optimality_gap(ub_obj,lb_obj)
        num_iteration_lb +=1
        improvement = prev_og-og
        print("current og")
        print(og)

print("Final Upper Bound Objective:", ub_obj)
print("Final Lower Bound Objective:", lb_obj)
print("Final Optimality Gap:", og)
last_lb_results_df = pd.DataFrame()

# Add node names
last_lb_results_df['Node Name'] = Nodes.iloc[:, 0]

# Add tech capacity decision (1 or 0) for each technology
for t in range(num_techs):
    last_lb_results_df[f'Tech_{t+1}_Decision'] = [
        1 if sum(last_tech_decision_lb.get((i, t, c), 0) for c in range(num_cap)) > 0 else 0
        for i in range(num_nodes)
    ]

# Add tech capacity at each node for every technology
for t in range(num_techs):
    last_lb_results_df[f'Tech_{t+1}_Capacity'] = [last_tech_capacity_lb.get((i, t), 0) for i in range(num_nodes)]

# Add detailed tech decisions for each technology and capacity
for t in range(num_techs):
    for c in range(num_cap):
        last_lb_results_df[f'Tech_{t+1}_Decision_Cap_{c+1}'] = [last_tech_decision_lb.get((i, t, c), 0) for i in range(num_nodes)]

# Save the results to an Excel file
last_lb_results_df.to_excel("C:\\Users\\smmashai\\Downloads\\Thesis\\master-shaik-main\\classic-model\\gsc_last_lb_solution_results100_1.5_05og.xlsx", index=False)
print("Last LB solution exported to 'last_lb_solution_results100.xlsx'")
# Export results to an Excel file
results_df = pd.DataFrame(results)
results_df.to_excel("C:\\Users\\smmashai\\Downloads\\Thesis\\master-shaik-main\\classic-model\\gsc_optimization_results_RTF100_1.5_05og.xlsx", index=False)
# Create tech_capacity_list for plotting
tech_capacity_list = {f"Tech_{t}_capacity": [] for t in range(num_techs)}
for t in range(num_techs):
    tech_capacity_list[f"Tech_{t}_capacity"] = [last_tech_capacity_lb.get((i, t), 0) for i in range(num_nodes) if last_tech_capacity_lb.get((i, t), 0) > 0]

# Plot technology distribution maps
for t in range(num_techs):
    fig, ax = plt.subplots(figsize=(9.25, 11.5))
    
    # Plot countries map
    countries = gpd.read_file(gpd.datasets.get_path("naturalearth_lowres"))
    countries[countries["name"] == "Germany"].plot(color="lightgrey", ax=ax)
    
    # Plot city locations if tech capacity list has non-zero values
    if tech_capacity_list[f"Tech_{t}_capacity"]:
        latitudes = [Nodes.iloc[i, 1] for i in range(num_nodes) if last_tech_capacity_lb.get((i, t), 0) > 0]
        longitudes = [Nodes.iloc[i, 2] for i in range(num_nodes) if last_tech_capacity_lb.get((i, t), 0) > 0]
        sizes = [last_tech_capacity_lb.get((i, t), 0) * multipliers[t] for i in range(num_nodes) if last_tech_capacity_lb.get((i, t), 0) > 0]
        
        ax.scatter(longitudes, latitudes, s=sizes, color='red', marker='o')
        ax.set_title(f"Tech {t+1} Capacity Distribution")
        ax.set_xlabel("Longitude")
        ax.set_ylabel("Latitude")
        ax.legend()
    
    # Save the figure
    fig.savefig(fr"C:\\Users\\smmashai\\Downloads\\Thesis\\master-shaik-main\\classic-model\\results\\LB\\tech{t}\\LB_final_tech2times{t}.jpg")
    plt.close(fig)
